In [44]:
# libraries
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.stattools import acf, adfuller
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.svm import SVR
from tqdm import tqdm
from collections import defaultdict
import holidays

# read in data

data downloaded from:

 - https://zenodo.org/record/5946808#.ZGNpddbP23I
 - https://zenodo.org/record/5841834#.ZGNpTNbP23J

### Wind data

##### Status data

In [45]:
# read in files
turbine_status = pd.read_csv('../data/Windturbinen/Kelmarsh/Status_Kelmarsh_1_2016-01-03_-_2017-01-01_228.csv',
                             skiprows = 9)

In [46]:
print(turbine_status.columns)
turbine_status.head()

Index(['Timestamp start', 'Timestamp end', 'Duration', 'Status', 'Code',
       'Message', 'Comment', 'Service contract category', 'IEC category'],
      dtype='object')


,Timestamp start,Timestamp end,Duration,Status,Code,Message,Comment,Service contract category,IEC category
0,2016-01-14 19:28:03,2016-01-23 14:36:32,211:08:29,Stop,111,Emergency stop nacelle,NaN,Emergency stop switch (Nacelle) (11),Forced outage
1,2016-01-14 19:28:03,2016-01-14 19:38:03,00:10:00,Warning,5720,Brake accumulator defect,NaN,Warnings (27),NaN
2,2016-01-14 19:28:05,2016-01-23 11:27:46,207:59:41,Informational,3835,Cable panel breaker open,NaN,Warnings (27),NaN
3,2016-01-14 19:28:05,2016-01-23 11:27:46,207:59:41,Informational,3830,Supply circuit breaker earthed,NaN,Warnings (27),Full Performance
4,2016-01-14 19:28:05,2016-01-23 14:09:18,210:41:13,Warning,3870,Overload transformer fan outlet air,NaN,Warnings (27),Full Performance


Status data is not relevant to this thesis !

##### Turbine Data

In [47]:
def reading_Windturbines (turbine_directory):
    # Columns to keep
    columns_to_keep = [
        '# Date and time',
        'Wind speed (m/s)',
        'Long Term Wind (m/s)',
        'Energy Export (kWh)'
    ]

    # Directory containing CSV files
    directory = f'../data/Windturbinen/{turbine_directory}/'

    # Dictionary to hold DataFrames for each turbine
    turbine_dataframes = defaultdict(list)

    # Get a list of CSV files in the directory
    csv_files = [f for f in os.listdir(directory) if f.startswith(f"Turbine_Data_{turbine_directory}_") and f.endswith(".csv")]

    # Iterate through the files in the directory with a tqdm progress bar
    for filename in tqdm(csv_files, desc='Processing files'):
        # Extract the turbine number from the filename
        turbine_number = filename.split("_")[3]  # Assuming the number is in this position

        # Read the CSV file, skipping the first 9 rows
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath, skiprows=9, usecols=columns_to_keep)

        # Convert the "Date and time" column to datetime
        df['# Date and time'] = pd.to_datetime(df['# Date and time'])

        # Append the DataFrame to the appropriate turbine's list
        turbine_dataframes[turbine_number].append(df)

    # Concatenate the DataFrames for each turbine
    for turbine_number, dfs in turbine_dataframes.items():
        turbine_dataframes[turbine_number] = pd.concat(dfs)
        turbine_dataframes[turbine_number].sort_values('# Date and time', inplace=True)
        turbine_dataframes[turbine_number] = turbine_dataframes[turbine_number].reset_index(drop=True)

    # Print the keys for the dictionary
    print("\n dictionary keys:")
    print(turbine_dataframes.keys())
    # print descriptive stuff for exemplary key
    print('\n Information for exemplary key:')
    first_key = list(turbine_dataframes.keys())[0]
    print('shape')
    print(turbine_dataframes[first_key].shape)
    print('\n missing values')
    print(turbine_dataframes[first_key].isna().sum())

    return turbine_dataframes

In [48]:
Kelmarsh_df = reading_Windturbines('Kelmarsh')

Processing files: 100%|██████████| 36/36 [00:12<00:00,  2.81it/s]


 dictionary keys:
dict_keys(['6', '4', '1', '3', '2', '5'])

 Information for exemplary key:
shape
(288864, 4)

 missing values
# Date and time            0
Wind speed (m/s)        9223
Long Term Wind (m/s)       0
Energy Export (kWh)     5072
dtype: int64


In [49]:
Penmanshiel_df = reading_Windturbines('Penmanshiel')

Processing files: 100%|██████████| 84/84 [00:45<00:00,  1.86it/s]


 dictionary keys:
dict_keys(['07', '08', '02', '05', '06', '15', '10', '14', '01', '04', '11', '12', '13', '09'])

 Information for exemplary key:
shape
(267014, 4)

 missing values
# Date and time            0
Wind speed (m/s)        5881
Long Term Wind (m/s)       0
Energy Export (kWh)     1032
dtype: int64


### Fraunhofer Data

In [50]:
# read in files
ENIT_2022_1 = pd.read_csv('../data/Fraunhofer/ENIT_1_2022.csv', sep=';', decimal=',')
ENIT_2022_2 = pd.read_csv('../data/Fraunhofer/ENIT_2_2022.csv', sep=';', decimal=',')
ENIT_2023_1 = pd.read_csv('../data/Fraunhofer/ENIT_1_2023.csv', sep=';', decimal=',')

# merge files
ENIT_df = pd.concat([ENIT_2022_1, ENIT_2022_2, ENIT_2023_1])

del ENIT_2022_1, ENIT_2022_2, ENIT_2023_1

In [66]:
# rename columns
ENIT_df.columns = ENIT_df.columns.str.replace('Wirkarbeit (Bezug) ', '').str.strip()

# add residual columns
ENIT_df['1.8 - Residual'] = ENIT_df.iloc[:, 1] - ENIT_df.iloc[:, 2:8].sum(axis=1)
ENIT_df['2.7 - Residual'] = ENIT_df.iloc[:, 9] - ENIT_df.iloc[:, 10:15].sum(axis=1)
ENIT_df['0.1 - Residual'] = ENIT_df.iloc[:, 16] - ENIT_df.iloc[:, 1] - ENIT_df.iloc[:, 9]

# change time to dateformat
ENIT_df['Zeitstempel'] = pd.to_datetime(ENIT_df['Zeitstempel'], format='%d.%m.%Y %H:%M:%S')

# reset index
ENIT_df = ENIT_df.reset_index(drop=True)

In [67]:
# data as dict with each company as a key
FraunhoferData = {
    'Hiller': ENIT_df
}

del ENIT_df

In [52]:
def FraunhoferData_descriptive (df):
    # Calculate the mean and standard deviation for each column
    mean = ENIT_df.iloc[:, 1:15].mean()
    std_dev = ENIT_df.iloc[:, 1:15].std()

    # Identify outliers using twice the standard deviation
    outliers = (ENIT_df.iloc[:, 1:15] < (mean - 2 * std_dev)) | (ENIT_df.iloc[:, 1:15] > (mean + 2 * std_dev))

    # Print the number of True values in each column
    print("Number of outliers in each column:")
    print(outliers.sum())

    # Get the row and column indices of the True values
    true_values_indices = outliers.where(outliers).stack().index

    # Print the row and column indices of the True values
    print("\nIndices of outliers:")
    for row, col in true_values_indices:
        print(f"Row: {row}, Column: {col}")

    # NA values
    print("\nNumber of missing Values:")
    print(ENIT_df.isna().sum())


Number of outliers in each column:
1.0 - Trafo 1 [Wh]                                   0
1.1 - Neubau, Wohnhaus, Holzplatz [Wh]               0
1.2 - Halle 3/1 Absaugung [Wh]                       0
1.3 - Halle 4/2 Maschinensaal/Tischfertigung [Wh]    0
1.4 - Halle 2/2 Verwaltung, Entwicklung [Wh]         0
1.5 - Halle 4/5 Lackieranlage [Wh]                   0
1.6 - Halle 1/6 Hausmeister [Wh]                     0
1.7 - Halle 3/3 Kompressor, Stuhlmontage [Wh]        0
2.0 - Trafo 2 [Wh]                                   0
2.1 - Halle 4/5 Schrankfertigung [Wh]                0
2.2 - Halle 2/4 Rilsan [Wh]                          0
2.3 - Halle 4/5 Stahlstuhl [Wh]                      0
2.4 - Halle 4/1 Schichtholz [Wh]                     0
2.5 - Halle 4/1 Absaugung Schichtholz [Wh]           0
dtype: int64

Indices of outliers:


# pre-processing pipeline

### outlier detection

In [69]:
def outlier_detection(data, dependent_var):
    # Only keep the specified columns
    df = data[[f'{dependent_var}']]

    # Calculate the mean and standard deviation for each column
    mean = df.mean()
    std_dev = df.std()

    # Calculate the z-scores for each value
    z_scores = (df - mean) / std_dev

    # Identify outliers using a threshold of 3 for z-scores
    outliers = (z_scores < -3) | (z_scores > 3)

    # Print the number of True values in each column
    print("Number of outliers in each column:")
    print(outliers.sum())

    # Get the row and column indices of the True values
    true_values_indices = outliers.where(outliers).stack().index

    # Print the row and column indices of the True values along with their z-scores
    print("\nIndices of outliers and their z-scores:")
    for row, col in true_values_indices:
        print(f"Row: {row}, Column: {col}, Z-Score: {z_scores.loc[row, col]}")

def Fraunhofer_descriptive (df):
    # Calculate the mean and standard deviation for each column
    mean = df.iloc[:, 1:15].mean()
    std_dev = df.iloc[:, 1:15].std()

    # Identify outliers using twice the standard deviation
    outliers = (df.iloc[:, 1:15] < (mean - 2 * std_dev)) | (df.iloc[:, 1:15] > (mean + 2 * std_dev))

    # Print the number of True values in each column
    print("Number of outliers in each column:")
    print(outliers.sum())

    # Get the row and column indices of the True values
    true_values_indices = outliers.where(outliers).stack().index

    # Print the row and column indices of the True values
    print("\nIndices of outliers:")
    for row, col in true_values_indices:
        print(f"Row: {row}, Column: {col}")

    # NA values
    print("\nNumber of missing Values:")
    print(df.isna().sum())

In [58]:
# column names as strings
outlier_detection(Kelmarsh_df['1'], 'Energy Export (kWh)')

Number of outliers in each column:
Energy Export (kWh)     6
Long Term Wind (m/s)    0
dtype: int64

Indices of outliers and their z-scores:
Row: 27407, Column: Energy Export (kWh), Z-Score: 3.2063598722064204
Row: 167992, Column: Energy Export (kWh), Z-Score: 3.835573415983272
Row: 175324, Column: Energy Export (kWh), Z-Score: 3.5877014138893606
Row: 234653, Column: Energy Export (kWh), Z-Score: 3.4923660284686258
Row: 236122, Column: Energy Export (kWh), Z-Score: 3.8260398774411986
Row: 247774, Column: Energy Export (kWh), Z-Score: 3.3016952576271557


In [70]:
Fraunhofer_descriptive(FraunhoferData['Hiller'])

Number of outliers in each column:
1.0 - Trafo 1 [Wh]                                   0
1.1 - Neubau, Wohnhaus, Holzplatz [Wh]               0
1.2 - Halle 3/1 Absaugung [Wh]                       0
1.3 - Halle 4/2 Maschinensaal/Tischfertigung [Wh]    0
1.4 - Halle 2/2 Verwaltung, Entwicklung [Wh]         0
1.5 - Halle 4/5 Lackieranlage [Wh]                   0
1.6 - Halle 1/6 Hausmeister [Wh]                     0
1.7 - Halle 3/3 Kompressor, Stuhlmontage [Wh]        0
2.0 - Trafo 2 [Wh]                                   0
2.1 - Halle 4/5 Schrankfertigung [Wh]                0
2.2 - Halle 2/4 Rilsan [Wh]                          0
2.3 - Halle 4/5 Stahlstuhl [Wh]                      0
2.4 - Halle 4/1 Schichtholz [Wh]                     0
2.5 - Halle 4/1 Absaugung Schichtholz [Wh]           0
dtype: int64

Indices of outliers:

Number of missing Values:
Zeitstempel                                             0
1.0 - Trafo 1 [Wh]                                     15
1.1 - Neubau, Woh

### stationarity test

don't run - takes ~10 minutes with 12 cores

In [59]:
%%script false --no-raise-error


# Assuming turbine_dataframes is a dictionary with your DataFrames
for turbine, df in Kelmarsh_df.items():
    full_series = df['Energy Export (kWh)']

    # Extract the last 75% of the series
    series = full_series[int(0.25 * len(full_series)):]
    
    # Perform the Augmented Dickey-Fuller test
    result = adfuller(series.dropna()) # Drop NaN values if necessary

    # Extract the p-value
    p_value = result[1]

    # Determine whether to reject the null hypothesis (non-stationary)
    if p_value < 0.05:
        print(f'{turbine}: Time series is likely stationary (p={p_value})')
    else:
        print(f'{turbine}: Time series is likely non-stationary (p={p_value})')

all time series have a p-value < 0.05 and are thus likely stationary.

Hence no additional steps have to be undertaken to use the time series at hand.

### Results table

In [60]:
# metrics function
def compute_metrics(df, column_name='Energy Export (kWh)'):
    # Number of values
    total_values = len(df[column_name])
    
    # Missing values
    missing_values = df[column_name].isna().sum()
    
    # Z-scores for outlier detection
    mean = df[column_name].mean()
    std_dev = df[column_name].std()
    z_scores = (df[column_name] - mean) / std_dev

    # Number of outliers and highest z-score
    outliers = z_scores[np.abs(z_scores) > 3]
    num_outliers = len(outliers)
    highest_z_score = outliers.abs().max() if num_outliers > 0 else 0

    return [total_values, missing_values, num_outliers, highest_z_score]

# apply function to df
summary_data = []

# Loop for Kelmarsh_df
for turbine, df in Kelmarsh_df.items():
    metrics = compute_metrics(df)
    summary_data.append(['Kelmarsh', turbine] + metrics)

# Loop for Penmanshiel_df
for turbine, df in Penmanshiel_df.items():
    metrics = compute_metrics(df)
    summary_data.append(['Penmanshiel', turbine] + metrics)

# Convert to a summary DataFrame
summary_df = pd.DataFrame(summary_data, columns=['Location', 'Turbine', 'Total Values', 'Missing Values', 'Outliers', 'Highest Z-Score'])

summary_df

,Location,Turbine,Total Values,Missing Values,Outliers,Highest Z-Score
0,Kelmarsh,6,288864,5072,4625,5.445566
1,Kelmarsh,4,288864,4938,2,3.370353
2,Kelmarsh,1,288864,3449,6,3.835573
3,Kelmarsh,3,288864,4195,3179,4.069864
4,Kelmarsh,2,288864,3041,5,3.636474
5,Kelmarsh,5,288864,3937,3091,4.793479
6,Penmanshiel,07,267014,1032,1,3.595320
7,Penmanshiel,08,259106,3977,1,3.302787
8,Penmanshiel,02,266923,2903,1,3.441052
9,Penmanshiel,05,265135,802,1,3.479419


### feature engineering Fraunhofer

trend and seasonal decomposition is not investigated, to make it harder for the Models to predict

In [61]:
FraunhoferData['Hiller'].head()

,Zeitstempel,1.0 - Trafo 1 [Wh],"1.1 - Neubau, Wohnhaus, Holzplatz [Wh]",1.2 - Halle 3/1 Absaugung [Wh],1.3 - Halle 4/2 Maschinensaal/Tischfertigung [Wh],"1.4 - Halle 2/2 Verwaltung, Entwicklung [Wh]",1.5 - Halle 4/5 Lackieranlage [Wh],1.6 - Halle 1/6 Hausmeister [Wh],"1.7 - Halle 3/3 Kompressor, Stuhlmontage [Wh]",2.0 - Trafo 2 [Wh],2.1 - Halle 4/5 Schrankfertigung [Wh],2.2 - Halle 2/4 Rilsan [Wh],2.3 - Halle 4/5 Stahlstuhl [Wh],2.4 - Halle 4/1 Schichtholz [Wh],2.5 - Halle 4/1 Absaugung Schichtholz [Wh],2.6 - Halle 3/6a Schlatter [Wh],Übergabezähler [Wh],1.8 - Residual,2.7 - Residual,0.1 - Residual
0,2022-01-01 00:15:00,2490.59,820.04,30.04,596.89,3064.04,0.11,136.98,205.81,3960.94,250.94,547.34,598.18,64.50,54.85,0.0,0.0,-2157.51,2445.13,-6451.53
1,2022-01-01 00:30:00,2416.07,677.48,29.32,651.33,3050.03,0.06,90.86,202.52,3872.87,279.26,537.12,571.65,63.96,53.31,0.0,0.0,-2083.01,2367.57,-6288.94
2,2022-01-01 00:45:00,2410.21,640.79,29.18,752.23,3020.57,0.10,92.29,194.06,3874.72,275.07,529.85,551.04,63.60,53.24,0.0,20000.0,-2124.95,2401.92,13715.07
3,2022-01-01 01:00:00,2525.85,715.91,29.27,612.53,3324.27,0.03,115.20,197.47,4035.20,276.31,537.56,553.50,63.96,53.54,0.0,0.0,-2271.36,2550.33,-6561.05
4,2022-01-01 01:15:00,2534.66,657.07,29.32,710.75,3322.08,0.04,110.17,202.56,4052.54,275.54,531.48,548.32,63.81,53.24,0.0,0.0,-2294.77,2580.15,-6587.20


In [62]:
# add time variables

# Extract day of the week
FraunhoferData['Hiller']['day_of_week'] = FraunhoferData['Hiller']['Zeitstempel'].dt.day_name()

# Extract season
def get_season(date):
    if (date.month == 3 and date.day >= 20) or (date.month == 4 or date.month == 5) or (date.month == 6 and date.day < 21):
        return 'Spring'
    elif (date.month == 6 and date.day >= 21) or (date.month == 7 or date.month == 8) or (date.month == 9 and date.day < 22):
        return 'Summer'
    elif (date.month == 9 and date.day >= 22) or (date.month == 10 or date.month == 11) or (date.month == 12 and date.day < 21):
        return 'Autumn'
    else:
        return 'Winter'

FraunhoferData['Hiller']['season'] = FraunhoferData['Hiller']['Zeitstempel'].apply(get_season)

# Get German public holidays for recent years
german_holidays = holidays.Germany(years=[2019, 2020, 2021, 2022, 2023])

# Create a new column indicating whether the date is a public holiday
FraunhoferData['Hiller']['public_holiday'] = FraunhoferData['Hiller']['Zeitstempel'].dt.date.isin(german_holidays).astype(int)

In [64]:
def time_features(df: pd.DataFrame, state: str = None) -> pd.DataFrame:
    # Ensure the eitstempel column is a datetime object
    df['Zeitstempel'] = pd.to_datetime(df['Zeitstempel'], format='%d.%m.%Y %H:%M:%S')

    # Add day of the week column
    df['day_of_week'] = df['Zeitstempel'].dt.day_name()

    # Add season column based on month
    def get_season(month):
        if 3 <= month <= 5:
            return "Spring"
        elif 6 <= month <= 8:
            return "Summer"
        elif 9 <= month <= 11:
            return "Autumn"
        else:
            return "Winter"

    df['season'] = df['Zeitstempel'].dt.month.apply(get_season)

    # Get German public holidays for recent years, including regional holidays for specified state
    years_to_consider = list(range(df['Zeitstempel'].dt.year.min(), df['Zeitstempel'].dt.year.max() + 1))
    german_holidays = holidays.Germany(years=years_to_consider, prov=state)

    # Create a new column indicating whether the date is a public holiday or weekend
    df['Weekday_end'] = df.apply(lambda row: "Weekend" if row['Zeitstempel'].date() in german_holidays or row['day_of_week'] in ['Saturday', 'Sunday'] else "Weekday", axis=1)

    return df

# Call the function with Bavaria as the state
time_features(FraunhoferData['Hiller'], "BY")[['Zeitstempel', 'day_of_week', 'season', 'Weekday_end']].head()


,Zeitstempel,day_of_week,season,Weekday_end
0,2022-01-01 00:15:00,Saturday,Winter,Weekend
1,2022-01-01 00:30:00,Saturday,Winter,Weekend
2,2022-01-01 00:45:00,Saturday,Winter,Weekend
3,2022-01-01 01:00:00,Saturday,Winter,Weekend
4,2022-01-01 01:15:00,Saturday,Winter,Weekend
...,...,...,...,...
21002,2023-08-07 22:45:00,Monday,Summer,Weekday
21003,2023-08-07 23:00:00,Monday,Summer,Weekday
21004,2023-08-07 23:15:00,Monday,Summer,Weekday
21005,2023-08-07 23:30:00,Monday,Summer,Weekday


# save files

In [ ]:
%store Kelmarsh_df Penmanshiel_df FraunhoferData

Stored 'Kelmarsh_df' (defaultdict)
Stored 'Penmanshiel_df' (defaultdict)
